# 数据基础篇
此篇核心记录一些常用表的核心字段信息。主要包含原始日志表，与汇总聚合表


## 离线日志表
大部分原始日志表都会包含几个核心信息，日志时间，请求ID，设备ID，IP，UA，VendorID, 广告位ID，DSPID等广告基础信息，不过在不同的日志表中有不同的命名，后面会具体描述。

### FTX
#### FTX请求表`dwd.d_ad_ftx_request`:
此表主要包含媒体的原始请求信息以及此次请求各家DSP的响应码，包含**全部**的请求日志，由于数据过大，目前只保留最近**15天**数据。

以下为核心常用字段描述
- 设备信息
    - mobile_os, mobile_os_version: 系统，及系统版本
    - mobile_brand, mobile_mode: 品牌机型
    - mobile_longitude, mobile_latitude: 经纬度
    - mobile_app_name, mobile_app_bundle_id, mobile_app_category, mobile_app_version: APP名称，ID，类型，版本
    - mobile_screen_width, mobile_screen_height: 屏幕尺寸
    - mobile_idfa, mobile_imei, mobile_oa_id, mobile_mac, mobile_aaid, mobile_caid: 设备ID系列，有些是原始值，有些是加密，具体分析
    - ftx_user_id: 泛为使用的ID，等于其它表中的ext_user_id
- 媒体信息
    - id: 请求ID，等同于曝光点击表中的ext_request_id。
    - source_id: 等同于其它表的vendor_id，这个名称会更通用些。
    - ad_slot_tag_id: 媒体广告位ID(adx_slot_id)。
    - ad_slot_type: 广告位类型，指代开屏，信息流，贴片等信息 @TODO 补全映射关系。
    - user_adx_rules: 媒体传过来的人群标签，比如QTT给我们传了淘宝安装包，会有一个媒体那边的标签信息，具体映射含义，需要运营与媒体确认。
- ADX服务端获取信息
    - timestamp: 请求时间
    - ip, geo_code: 请求IP，及广协IP库的城市编码
    - user_agent: 请求header中的user_agent信息，与同一次曝光机会的曝光、点击的ua不一定一样
- DSP信息：
    - dsp_ids_array: DSP的响应信息，包含dsp id, ftx_order_id, 以及响应code。@TODO 补全code的含义

#### FTX响应表`dwd.d_ad_ftx_response`
此表主要包含各DSP对请求的响应详细信息，每个DSP的响应都会产生一行数据，即数据行数会比请求数据行数多，包含**全部**的请求日志，由于数据过大，目前只保留最近**15天**数据。

- 常规基础信息
    - timestamp: 响应时间
    - dsp_id, vendor_id, fancy_user_id
- 参竞信息：
    - error_code: 等于0代表DSP参竞，并且在FTX内部竞价胜出，最后会吐给媒体方。
    - seat_bid_custom_dsp_bid_price_orig: DSP的原始出价
    - seat_bid_price: FTX向媒体的出价
    - seat_bid_imp_monitor_urls: 曝光监测地址
    - seat_bid_clk_monitor_urls: 点击监测地址
    - seat_bid_landing_page: 落地页地址
    - landing_desc: 假如有落地页信息则在这个字段中

以下开始为广告的展点消相关日志表，他们包含一大部分的含义相同的基础信息字段，以下统一做个描述

- timestamp: 曝光，点击，唤起等触发时间，具体含义以其出现的表含义为准
- ext_timestamp: 展点对应的请求时间
- ip, geo_code: 展点获取的IP，及对应广协IP库的城市编码
- ext_ip, ext_geo_id: 展点对应的请求IP，及对应的广协IP库的城市编码
- user_agent, referer:  服务器在曝光的请求Header中user_agent, referer
- ua_id_v2: FTX对user_agent的编码，核心包含品牌机型系统版本信息。
- ext_request_id: 请求ID
- ext_vendor_id: 媒体ID
- ext_user_id: 设备ID
- ext_dsp_id: DSP ID
- ext_order_id: FTX系统的子订单
- ext_parent_order_id: FTX系统的父订单，可能为空
- ext_slot_id: 媒体的广告位ID
- ext_ftx_slot_id: 媒体广告位在FTX系统中的映射ID，不一定全部都有值 

####  FTX胜出表`dwd.d_ad_ftx_win_data` 
此表包含媒体的胜出数据，在国内的RTB行业中，此win并非在ADX/SSP的服务端的胜出，而是代表在服务端胜出并且再客户端成功展示，实际上与曝光数一致，相较与曝光表，它多出了一些RTB结算价格信息。数据至少保留一年

- 结算信息：
    - cost_price: 与媒体的成本价格，可以是媒体返回的RTB二价，也可以是系统中设置的固定价
    - charge_price: 与DSP的结算收入价格，其中FancyDSP，charge_price=cost_price
    - ext_dsp_bid_price: DSP响应时给的参竞价格。
    - ext_bid_price: FTX给媒体响应是的参竞价格。

  
#### FTX曝光表`dwd.d_ad_ftx_impression_data`
此表是广告在客户端成功展现的曝光数据。数据至少保留一年

- 特殊字段
    - anonymous_ua: 无来源识别 0：有来源 1：无来源  
    - ext_adx_bid_tag: 趣头条订单标签 分为 pdb pd   
    - ext_bid_opt_tag: 趣头条补量标签 带'opt'为补量  
    - ext_dsp_slot:用于查看实时usop后发给的dsp 的广告位信息  

#### FTX点击表`dwd.d_ad_ftx_click_data`
此表是客户端对广告的点击行为数据。数据至少保留一年

- 特殊字段
    - ignore_rule：反作弊过滤字段 0为正常点击  
    - clk_up_x_point，clk_up_y_point：点击相对于广告位区域的坐标  
    
 
#### FTX唤起表 `dwd.d_ad_ftx_dp_track_log`
媒体当客户点击广告后，唤起对应APP的回调数据，由于是媒体客户端判定目标APP有安装并尝试唤起时回调，并非在真正成功唤起时回调，估数据比真实吊起的数据要多，只能是参考数据。目前只有个别媒体支持，比如QTT，韩剧TV，人人视频。

#### FTX大宽表`dws.s_ad_ftx_join`
一份把上述的请求，响应，胜出，曝光，点击，吊起关联起来的数据，它有以下几个特征

- 数据为T+1
- 只保留了FTX最后参竞的数据
- 去掉了FancyDSP的数据
- 数据保留三个月


### DSP

#### 数据库 dwd

请求响应表 d_ad_ftx_request

曝光表 d_ad_impression

点击表 d_ad_clicklog_day

行为表1 d_ad_action

行为表2 d_ad_data_analysis_log
#### 数据库 dws

大宽表 s_ad_dsp_join

### 趣头条GD

#### 数据库 dwd

曝光表 dspv2_imp_data

点击表 dspv2_click_data

### APP安装包（趣头条提供）

#### 数据库 dwd

人群装机信息表 qtt_user_profile



## DB（mysql）

### FTX

#### 数据库 ftx

订单信息表 order

订单与amo关联表 order_detail_amo_order

dsp信息表 dsp

FTX广告位信息表 slot

#### 数据库 ftx_report

订单维度消耗表 ftx_order_consume_report

订单维度五分钟消耗表 data_5min_ftx_order_param_report

点击反作弊表 ftx_click_data

### DSP

#### 数据库 ad_fancy

投放信息表 ad

投放组信息表 ad_group

点位信息表 spot

广告主信息表 advertiser

投放计划信息表 campaign

投放计划与amo关联表 campaign_amo_order_mapping

#### 数据库 ad_respot

五分钟维度表 data_5min_aggr_sim

点击反作弊表 data_clk_antispam_report

#### 数据库 rpt_fancy

dsp 5min多维度实时表 rpt_advertiser